In [4]:
# -*- coding: utf-8 -*-
from JupyterNotebookUtil import JupyterNotebookUtil
from pathlib import Path 
from IPython.display import display, HTML
import IPython.display as ipd
from collections import OrderedDict
import os
import torch

GT_path = "../../save/groundtruth/mackenzie_norm/vocal/phoneme_results"
samples_path = "../../save/samples/inpainting/mackenzie_unet_diffusion_inpainter_audio"
gt_csv_path = "../../save/groundtruth/mackenzie_norm/vocal/phoneme_results/results/results.csv"
result_jsons_path = []
result_csvs_path = [gt_csv_path]
for root, dirs, files in os.walk(samples_path):
    if Path(root).stem == 'results' and '39' in root:
        for file in files:
            if Path(file).name == 'results_non_silent.csv':
                result_csvs_path.append(Path(root) / file)
    elif Path(root).stem == 'vocal_json_' and '39' in root:
        for file in files:
            if '_same_name_False.json' in Path(file).name:
                result_jsons_path.append(Path(root) / file)
print(result_csvs_path[2], result_jsons_path[2])

../../save/samples/inpainting/mackenzie_unet_diffusion_inpainter_audio/noise_True_inpainting_ratio_1.0/2024-05-31-11-27-56_scoreSDE/39/40_1/phoneme_results/results/results_non_silent.csv ../../save/samples/inpainting/mackenzie_unet_diffusion_inpainter_audio/noise_True_inpainting_ratio_1.0/2024-05-31-11-27-56_MCG/39/200_1/vocal_json_/195000_same_name_False.json


In [9]:
import json
import numpy as np
dict_list = []
        
for csv_path in result_csvs_path:
    inference = OrderedDict()
    phon_var, phon_acc = [], []
    # phon_var, phon_acc = 0, 0
    with open(csv_path, 'r') as f:
        result_list = json.load(f)
    for result in result_list:
        phon_var.append(result["var_"])
        phon_acc.append(result["acc_"])
    phon_var_mean = np.mean(phon_var)
    phon_var_std = np.std(phon_var)
    phon_acc_mean = np.mean(phon_acc)
    phon_acc_std = np.std(phon_acc)
    print(f"{str(csv_path).split('/')[6:10]}: var - {phon_var_mean:.4f} ± {phon_var_std:.4f}, acc - {phon_acc_mean:.4f} ± {phon_acc_std:.4f}")
    if str(csv_path).split('/')[6].split('_')[-1] == 'results':
        inference['set'] = 'groundtruth'
        inference['inpainting_ratio'] = '-'
        inference['inpaint_method'] = '-'
    else:
        inference['set'] = 'target'
        inference['inpainting_ratio'] = str(csv_path).split('/')[6].split('_')[-1] 
        inference['inpaint_method'] = str(csv_path).split('/')[7].split('_')[-1] 
        inference['diffusion_step'] = str(csv_path).split('/')[9].split('_')[0] 
        inference['resampling_step'] = str(csv_path).split('/')[9].split('_')[1] 
    inference['var_mean'] = phon_var_mean
    # inference['var_std'] = phon_var_std
    inference['acc_mean'] = phon_acc_mean
    # inference['acc_std'] = phon_acc_std
    dict_list.append(inference)

for json_path in result_jsons_path:
    inference = OrderedDict()
    with open(json_path, 'r') as f:
        result_list = json.load(f)
    fd = result_list['frechet_distance']
    fad = result_list['frechet_audio_distance']
    is_mean = result_list['inception_score_mean']
    is_std = result_list['inception_score_std']
    print(f"{str(json_path).split('/')[6:10]}: frechet_distance - {fd:.4f}, frechet_audio_distance - {fad:.4f}, inception_score - {is_mean:.4f} ± {is_std:.4f}")
    for dicts in dict_list:
        if dicts['inpainting_ratio'] == str(json_path).split('/')[6].split('_')[-1] and dicts['inpaint_method'] == str(json_path).split('/')[7].split('_')[-1] and dicts['diffusion_step'] == str(json_path).split('/')[9].split('_')[0] and dicts['resampling_step'] == str(json_path).split('/')[9].split('_')[1]:
            dicts['frechet_distance'] = fd
            dicts['frechet_audio_distance'] = fad
            # dicts['inception_score_mean'] = is_mean
            # dicts['inception_score_std'] = is_std
    for dicts in dict_list:
        if dicts['set'] == 'groundtruth':
            dicts['diffusion_step'] = '-'
            dicts['resampling_step'] = '-'
            dicts['frechet_distance'] = '-'
            dicts['frechet_audio_distance'] = '-'
            # dicts['inception_score_mean'] = '-'
            # dicts['inception_score_std'] = '-'

['phoneme_results', 'results', 'results.csv']: var - 3.1486 ± 0.5366, acc - 1.5317 ± 0.2282
['noise_True_inpainting_ratio_1.0', '2024-05-31-11-27-56_scoreSDE', '39', '200_1']: var - 3.4752 ± 0.6385, acc - 1.9596 ± 0.1728
['noise_True_inpainting_ratio_1.0', '2024-05-31-11-27-56_scoreSDE', '39', '40_1']: var - 3.4699 ± 0.5971, acc - 1.9567 ± 0.1575
['noise_True_inpainting_ratio_1.0', '2024-05-31-11-27-56_MCG', '39', '200_1']: var - 3.6583 ± 0.4034, acc - 2.0307 ± 0.1644
['noise_True_inpainting_ratio_1.0', '2024-05-31-11-27-56_MCG', '39', '40_1']: var - 3.2499 ± 0.5510, acc - 2.0039 ± 0.2247
['noise_True_inpainting_ratio_1.0', '2024-05-31-11-27-56_repaint', '39', '40_5']: var - 3.0890 ± 0.8698, acc - 1.8566 ± 0.2474
['noise_True_inpainting_ratio_0.75', '2024-05-31-11-27-56_scoreSDE', '39', '200_1']: var - 3.5301 ± 0.5408, acc - 1.9840 ± 0.2021
['noise_True_inpainting_ratio_0.75', '2024-05-31-11-27-56_scoreSDE', '39', '40_1']: var - 3.4846 ± 0.5916, acc - 1.9811 ± 0.1756
['noise_True_inpai

In [10]:
html_list = JupyterNotebookUtil.dict_list_to_html(dict_list)
import pandas as pd
# pd.set_option('display.max_colwidth', 500)
display(HTML(html_list))

set,inpainting_ratio,inpaint_method,var_mean,acc_mean,diffusion_step,resampling_step,frechet_distance,frechet_audio_distance
groundtruth,-,-,3.148567,1.531660,-,-,-,-
target,1.0,scoreSDE,3.475171,1.959580,200,1,17.369577,5.407567
target,1.0,scoreSDE,3.469942,1.956735,40,1,17.213609,5.127676
target,1.0,MCG,3.658278,2.030668,200,1,18.278001,5.341719
target,1.0,MCG,3.249908,2.003949,40,1,25.436109,6.351792
target,1.0,repaint,3.088986,1.856619,40,5,20.969882,4.798156
target,0.75,scoreSDE,3.530091,1.984037,200,1,18.406412,5.856203
target,0.75,scoreSDE,3.484639,1.981106,40,1,18.832271,5.562365
target,0.75,MCG,3.712670,2.048502,200,1,19.917215,6.137634
target,0.75,MCG,3.302112,1.968051,40,1,29.666236,8.367879


In [7]:


MAX_WAV_VALUE = 32768.0
mel_basis = {}
hann_window = {}

def load_wav(full_path):
    sampling_rate, data = read(full_path)
    return data, sampling_rate

def dynamic_range_compression_torch(x, C=1, clip_val=1e-5):
    return torch.log(torch.clamp(x, min=clip_val) * C)

def spectral_normalize_torch(magnitudes):
    output = dynamic_range_compression_torch(magnitudes)
    return output

def mel_spectrogram(y, n_fft, num_mels, sampling_rate, hop_size, win_size, fmin, fmax, center=False):
    if torch.min(y) < -1.:
        print('min value is ', torch.min(y))
    if torch.max(y) > 1.:
        print('max value is ', torch.max(y))

    global mel_basis, hann_window
    if fmax not in mel_basis:
        mel = librosa_mel_fn(sampling_rate, n_fft, num_mels, fmin, fmax)
        mel_basis[str(fmax)+'_'+str(y.device)] = torch.from_numpy(mel).float().to(y.device)
        hann_window[str(y.device)] = torch.hann_window(win_size).to(y.device)

    y = torch.nn.functional.pad(y.unsqueeze(1), (int((n_fft-hop_size)/2), int((n_fft-hop_size)/2)), mode='reflect')
    y = y.squeeze(1)

    spec = torch.stft(y, n_fft, hop_length=hop_size, win_length=win_size, window=hann_window[str(y.device)],
                      center=center, pad_mode='reflect', normalized=False, onesided=True, return_complex=True)

    # spec = torch.sqrt(spec.pow(2).sum(-1)+(1e-9)) # for previous version of pytorch
    spec = torch.sqrt(spec.abs()+(1e-9))

    spec = torch.matmul(mel_basis[str(fmax)+'_'+str(y.device)], spec)
    spec = spectral_normalize_torch(spec)

    return spec

def make_dict_list(inference_folders):
    dict_list = []
    for folder in inference_folders:
        inference = OrderedDict() # OrderedDict([('ckpts',''), ('Alto-5??/0034.wav',''), ('Bass-1??/0023.wav',''), ...])
        inference['0_ckpt'] = str(folder.stem)
        if 'predictions' in str(folder):
            objectives = str(folder).replace('predictions', 'objectives') + '.txt'
            with open(objectives, 'r') as f:
                avg_mss_loss, _, fad_score = tuple(f.read().splitlines()[-3:])

        if 'groundtruth' in str(folder):
            inference['1_FAD'] = '-'
            inference['2_mss_loss'] = '-'
        else:
            inference['1_FAD'] = round(float(fad_score), 3)
            inference['2_mss_loss'] = round(float(avg_mss_loss), 3)
        for wav_path in folder.glob('**/*.wav'):
            inference[str(wav_path.parent.stem + '/' + wav_path.stem)] = [JupyterNotebookUtil.get_html_from_src_path(type="audio", path=str(wav_path), width=300),  JupyterNotebookUtil.get_html_from_src_path(type="img", path=str(wav_path).replace('.wav', '.png'), width=300)]
            # inference[str(wav_path.parent.stem + '/' + wav_path.stem) + '_mel'] = JupyterNotebookUtil.get_html_from_src_path(type="img", path=str(wav_path).replace('.wav', '.png'), width=150)
        dict_list.append(OrderedDict(sorted(inference.items(), key=lambda x: x[0])))
        dict_list = sorted(dict_list, key= lambda x: x['0_ckpt'])
    return dict_list
dict_list = make_dict_list(entire_list)
# print(dict_list)
html_list = JupyterNotebookUtil.dict_list_to_html(dict_list)
# print(html_list)

NameError: name 'entire_list' is not defined

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 500)
display(HTML(html_list))